In [1]:
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from rich import print

In [2]:
class LocalWindFieldDataset(Dataset):
    def __init__(self, csv_file, rect_fol, wind_fol, root_dir="", local=32):
        self.main_data = pd.read_csv(os.path.join(root_dir, csv_file))
        # generate the local wind field using the local and main_data parameter
        self.rect_file_path = os.path.join(root_dir, rect_fol)
        self.wind_file_path = os.path.join(root_dir, wind_fol)
        
    def __len__(self):
        return len(self.main_data)
    
    def __getitem__(self, idx):
        data = self.main_data.iloc[idx]
        rect_path = os.path.join(self.rect_file_path, str(data["index"]) + "_r.npy")
        wind_path = os.path.join(self.wind_file_path, str(data["index"]) + "_m.npy")
        rect = np.load(rect_path)
        wind = np.load(wind_path)
        lidar = 0
        # using data generate lidar scan 
        return lidar,rect, wind[0][0]   

In [3]:
local_wind_field_dataset = LocalWindFieldDataset(csv_file="data.csv",rect_fol = 'rects',wind_fol = 'wind_fields', root_dir = 'data',local=32)

In [4]:
for i, sample in enumerate(local_wind_field_dataset, 1):
    rect, wind = sample
    print(f"{i}: \n {data}")
    print(f"Rectangles - \n {rect} \n \n Wind Fields- \n {wind}")

1: 
 index     1
xr       20
xy       30
Name: 0, dtype: int64

Rectangles - 
 [[[  1   2]
  [  3   4]]

 [[  5   6]
  [  1   3]]

 [[ 10 100]
  [ 12 120]]] 
 
 Wind Fields- 
 [[[49 10]
  [83 95]
  [18 71]]

 [[26 73]
  [18  0]
  [57 43]]

 [[77 90]
  [42 90]
  [21 79]]

 [[79 41]
  [17 77]
  [15 29]]]

2: 
 index     2
xr       30
xy       40
Name: 1, dtype: int64

Rectangles - 
 [[[  10   20]
  [  30   40]]

 [[  50   60]
  [  10    0]]

 [[ 100 1000]
  [ 102 1200]]] 
 
 Wind Fields- 
 [[[27 87]
  [75 30]
  [71 19]]

 [[91 60]
  [98 89]
  [22 18]]

 [[89 88]
  [69 91]
  [16 25]]

 [[84 29]
  [17 91]
  [58 35]]]

3: 
 index     3
xr       40
xy       50
Name: 2, dtype: int64

Rectangles - 
 [[[ 0  0]
  [30 10]]

 [[50 40]
  [10 40]]

 [[ 0 40]
  [ 2 12]]

 [[50 40]
  [10 40]]

 [[50  0]
  [10  4]]] 
 
 Wind Fields- 
 [[[78 32]
  [ 6 10]
  [ 8 46]]

 [[36 77]
  [11 53]
  [90  0]]

 [[68 37]
  [90 98]
  [98 72]]

 [[40 94]
  [60 51]
  [65 29]]]